<a href="https://colab.research.google.com/github/PadmarajBhat/Rapids.AI/blob/master/rapids_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Sanity Check #

Click the _Runtime_ dropdown at the top of the page, then _Change Runtime Type_ and confirm the instance type is _GPU_.

Check the output of `!nvidia-smi` to make sure you've been allocated a Tesla T4.

In [1]:
!nvidia-smi

Wed Jul 17 23:09:36 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    17W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import pynvml

pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
device_name = pynvml.nvmlDeviceGetName(handle)
print(device_name)
if device_name != b'Tesla T4':
  raise Exception("""
    Unfortunately this instance does not have a T4 GPU.
    
    Please make sure you've configured Colab to request a GPU instance type.
    
    Sometimes Colab allocates a Tesla K80 instead of a T4. Resetting the instance.

    If you get a K80 GPU, try Runtime -> Reset all runtimes...
  """)
else:
  print('Woo! You got the right kind of GPU!')

b'Tesla T4'
Woo! You got the right kind of GPU!


#Setup:

1. Install most recent Miniconda release compatible with Google Colab's Python install  (3.6.7)
2. Install RAPIDS libraries
3. Set necessary environment variables
4. Copy RAPIDS .so files into current working directory, a workaround for conda/colab interactions

In [11]:
# intall miniconda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

# install RAPIDS packages
!conda install -q -y --prefix /usr/local -c conda-forge \
  -c rapidsai-nightly/label/cuda10.0 -c nvidia/label/cuda10.0 \
  cudf cuml


!conda install -q -y --prefix /usr/local -c nvidia -c rapidsai \
  -c numba -c conda-forge -c defaults nvstrings=0.8 python=3.6 cudatoolkit=10.0

# set environment vars
import sys, os, shutil
sys.path.append('/usr/local/lib/python3.6/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

# copy .so files to current working dir
for fn in ['libcudf.so', 'librmm.so']:
  shutil.copy('/usr/local/lib/'+fn, os.getcwd())

--2019-07-17 23:16:56--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6.7-hc745277_3 ...
installing: xz-5.2.4-h14c3975_4 ...
installing: yaml-0.1.7-had09818_2 ...
installing: zlib-1.2.11-ha838bed_2 ...
installing: libedit-3.1.20170329-h6b74fdf_2 

In [0]:
#!pip install nvstrings-cuda100
#!echo "New LD_LIBRARY_PATH :"$LD_LIBRARY_PATH
#!set LD_LIBRARY_PATH="/usr/lib64-nvidia:/usr/local/cuda-10.0/lib64/"
#!echo "New LD_LIBRARY_PATH :"$LD_LIBRARY_PATH

     |████████████████████████████████| 9.1MB 2.6MB/s 
New LD_LIBRARY_PATH :/usr/lib64-nvidia


In [0]:
#!conda install -c nvidia -c rapidsai -c numba -c conda-forge -c defaults \
#    nvstrings=0.8 python=3.6 cudatoolkit=10.0

In [0]:
#import os
#os.environ['LD_LIBRARY_PATH'] = "$LD_LIBRARY_PATH:/usr/local/cuda/lib64/"
#!echo "New LD_LIBRARY_PATH :"$LD_LIBRARY_PATH

In [6]:
!ls /usr/local/cuda/lib64/*9.2

ls: cannot access '/usr/local/cuda/lib64/*9.2': No such file or directory


In [7]:
!ls /usr/lib64-nvidia/*art*.so
!cp /usr/local/cuda/lib64/lib*art*.so /usr/lib64-nvidia/
!cp /usr/lib64-nvidia/*art*.so /usr/lib64-nvidia/libcudart.so.9.2
!ls /usr/lib64-nvidia/*art*


ls: cannot access '/usr/lib64-nvidia/*art*.so': No such file or directory
/usr/lib64-nvidia/libcudart.so	/usr/lib64-nvidia/libcudart.so.9.2


# cuDF and cuML Examples #

Now you can run code! 

What follows are basic examples where all processing takes place on the GPU.

#[cuDF](https://github.com/rapidsai/cudf)#

Load a dataset into a GPU memory resident DataFrame and perform a basic calculation.

Everything from CSV parsing to calculating tip percentage and computing a grouped average is done on the GPU.

_Note_: You must import nvstrings and nvcategory before cudf, else you'll get errors.

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [9]:
!ls -l "/content/drive/My Drive/Colab Notebooks/TrainData_PA.csv"

-rw------- 1 root root 5493629 Jul 16 23:19 '/content/drive/My Drive/Colab Notebooks/TrainData_PA.csv'


In [12]:
import nvstrings, nvcategory, cudf
import io, requests
tips_df = cudf.read_csv("/content/drive/My Drive/Colab Notebooks/TrainData_PA.csv")
tips_df.head()

<cudf.DataFrame ncols=40 nrows=5 >

In [19]:
print(tips_df.head(), tips_df.shape, tips_df.columns)

   county        city  zipcode                            address  state  rent            latitude ...  HomePrice
0    None     WEXFORD            266 Clematis Dr Allegheny County     PA  2400             40.6182 ...     158051
1    None   WHITEHALL                2310 N 1st Ave Lehigh County     PA   995           40.649906 ...     158051
2    None   WHITEHALL           3338 St Stephens Ln Lehigh County     PA  1740           40.646282 ...     158051
3    None  WAYNESBORO                97 W Main St Franklin County     PA   675  39.756992000000004 ...     158051
4    None  QUAKERTOWN                 200 E Broad St Bucks County     PA  1300  40.441176999999996 ...     158051
[32 more columns] (18203, 40) Index(['county', 'city', 'zipcode', 'address', 'state', 'rent', 'latitude',
       'longitude', 'cemetery_dist_miles', 'nationalhighway_miles',
       'railline_miles', 'starbucks_miles', 'walmart_miles', 'hospital_miles',
       'physician_dist_miles', 'dentist_dist_miles', 'opt_dist_

In [30]:
print(tips_df[tips_df.columns[:-1]])

    county         city  zipcode                            address  state  rent            latitude ...  Crime_Rate
0    None      WEXFORD            266 Clematis Dr Allegheny County     PA  2400             40.6182 ...         2.4
1    None    WHITEHALL                2310 N 1st Ave Lehigh County     PA   995           40.649906 ...         2.4
2    None    WHITEHALL           3338 St Stephens Ln Lehigh County     PA  1740           40.646282 ...         2.4
3    None   WAYNESBORO                97 W Main St Franklin County     PA   675  39.756992000000004 ...         2.4
4    None   QUAKERTOWN                 200 E Broad St Bucks County     PA  1300  40.441176999999996 ...         2.4
5    None   WAYNESBORO           407 Viewpoint Way Franklin County     PA  1025  39.766594000000005 ...         2.4
6    None   WAYNESBORO           403 Viewpoint Way Franklin County     PA  1025  39.766580000000005 ...         2.4
7    None   WAYNESBORO                240 Crown Ct Franklin County     

In [24]:
print(tips_df.groupby('city').HomePrice.mean().reset_index())

              city           HomePrice
0        ABINGTON   165602.2857142857
1        AIRVILLE            158051.0
2           AKRON  170204.57142857142
3  ALBRIGHTSVILLE  111965.84615384616
4        ALBURTIS  194317.18181818182
5       ALIQUIPPA   90321.06666666667
6       ALLENTOWN  144091.08250825084
7    ALLISON PARK  185304.86956521738
8         ALTOONA  141058.81818181818
9          AMBLER           259464.06
[664 more rows]


In [31]:
from cuml import SGD

sgd = SGD(eta0=0.1)

result_sgd = sgd.fit(tips_df[tips_df.columns[:-1]], tips_df[tips_df.columns[-1]])

ValueError: ignored

In [0]:
!ls  /usr/local/cuda/lib64/*art.so

/usr/local/cuda/lib64/libcudart.so


In [0]:
import nvstrings, nvcategory, cudf
import io, requests

# download CSV file from GitHub
url="https://github.com/plotly/datasets/raw/master/tips.csv"
content = requests.get(url).content.decode('utf-8')

# read CSV from memory
tips_df = cudf.read_csv(io.StringIO(content))
tips_df['tip_percentage'] = tips_df['tip']/tips_df['total_bill']*100

# display average tip by dining party size
print(tips_df.groupby('size').tip_percentage.mean())

size
1     21.72920154872781
2    16.571919173482893
3    15.215685473711831
4    14.594900639351332
5    14.149548965142023
6    15.622920072028379
Name: tip_percentage, dtype: float64


#[cuML](https://github.com/rapidsai/cuml)#

This snippet loads a 

As above, all calculations are performed on the GPU.

In [0]:
import cuml

# Create and populate a GPU DataFrame
df_float = cudf.DataFrame()
df_float['0'] = [1.0, 2.0, 5.0]
df_float['1'] = [4.0, 2.0, 1.0]
df_float['2'] = [4.0, 2.0, 1.0]

# Setup and fit clusters
dbscan_float = cuml.DBSCAN(eps=1.0, min_samples=1)
dbscan_float.fit(df_float)

print(dbscan_float.labels_)

0    0
1    1
2    2
dtype: int32


# Next Steps #

For an overview of how you can access and work with your own datasets in Colab, check out [this guide](https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92).

For more RAPIDS examples, check out our RAPIDS notebooks repos:
1. https://github.com/rapidsai/notebooks
2. https://github.com/rapidsai/notebooks-extended